## Conversation Q&A Chatbot 

- In many Q&A applcations we want to allow the user to have a back and froth conversation meaning the application need some sort of memort of past questions and answers and some login for incorporating tose into its current things 

- In this guide we focus on adding logic for incorporating historical messages further detials on chat history management


- we Will cover two approches : 
    - Chains , in which we always execute a retrieval step 
    - Agents , in which we give an LLM descretion over whether and how to execute  a retrival step 

    

In [22]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [23]:
from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    api_key=groq_api_key)

llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7a5dd4a0ef80>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7a5dd4a0d420>, model_name='deepseek-r1-distill-llama-70b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [24]:
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2", model_kwargs={"device": "cuda"})


In [25]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [26]:
# 1. load chunk and index the contents of the blog to create a reriever 

import bs4

loader = WebBaseLoader(
    web_paths = ("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs = dict(
        parse_only = bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        ),
        
    )
)

docs = loader.load()
docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistake

In [27]:
text_splitter  = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    
)
texts = text_splitter.split_documents(docs)
db = Chroma.from_documents(
    texts,
    embeddings,
    collection_name = "lilianweng_agent",
    persist_directory = "chroma_db"
)
retriever = db.as_retriever(
    search_kwargs = {
        "k": 5
    }
)

In [28]:
## Prompt template 
system_prompt = (
    "You are a helpful assistant for question-answering tasks. "
    "use the following pices of retrived information to answer the question. "
    "If you don't know the answer, just say that you don't know. "
    "Do not try to make up an answer. "


    "\n\n"

    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [29]:
question_anser_chain = create_stuff_documents_chain(llm, prompt=prompt) ## combin all doc and pass to context
rag_chain = create_retrieval_chain(retriever,question_anser_chain)

In [30]:
response  = rag_chain.invoke(
    {
        "input": "What is the difference between a tool and an agent?",
       
    }
)
response

{'input': 'What is the difference between a tool and an agent?',
 'context': [Document(id='addec3c6-66f9-419e-8fdb-610d67757f04', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='This benchmark evaluates the agent’s tool use capabilities at three levels:\n\nLevel-1 evaluates the ability to call the API. Given an API’s description, the model needs to determine whether to call a given API, call it correctly, and respond properly to API returns.\nLevel-2 examines the ability to retrieve the API. The model needs to search for possible APIs that may solve the user’s requirement and learn how to use them by reading documentation.\nLevel-3 assesses the ability to plan API beyond retrieve and call. Given unclear user requests (e.g. schedule group meetings, book flight/hotel/restaurant for a trip), the model may have to conduct multiple API calls to solve it.'),
  Document(id='f8adb792-d0ad-4159-9f18-3662e42e804a', metadata={'source': 'https://lilianwen

In [31]:
response['answer']

"<think>\nOkay, so I need to figure out the difference between a tool and an agent based on the provided information. Let me read through the given text carefully to see if I can find any clues or definitions that distinguish between the two.\n\nThe text talks about LLM-powered autonomous agents and their capabilities. It mentions that the agent uses tools like APIs and memory. For example, in Level-1, the agent calls an API correctly and handles the response. Level-2 is about retrieving the right API, and Level-3 involves planning multiple API calls for complex tasks.\n\nThe agent also has components like planning (subgoal decomposition, reflection) and memory. It uses natural language as an interface between the LLM and external components, but there are reliability issues with model outputs, such as formatting errors or refusing instructions.\n\nSo, from this, it seems that a tool is something the agent uses to perform tasks, like an API or memory. The agent, on the other hand, is t

# Add chain history

In [32]:
response  = rag_chain.invoke(
    {
        "input": "so give me example",
       
    }
)
response # not add a history

{'input': 'so give me example',
 'context': [Document(id='b73d58c3-b0c0-448e-8d85-2e968db2221a', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Memory stream: is a long-term memory module (external database) that records a comprehensive list of agents’ experience in natural language.\n\nEach element is an observation, an event directly provided by the agent.\n- Inter-agent communication can trigger new natural language statements.\n\n\nRetrieval model: surfaces the context to inform the agent’s behavior, according to relevance, recency and importance.\n\nRecency: recent events have higher scores\nImportance: distinguish mundane from core memories. Ask LM directly.\nRelevance: based on how related it is to the current situation / query.\n\n\nReflection mechanism: synthesizes memories into higher level inferences over time and guides the agent’s future behavior. They are higher-level summaries of past events (<- note that this is a bit differen

In [33]:
from langchain.chains import create_history_aware_retriever # retriver also aware of the history
from langchain_core.prompts import MessagesPlaceholder


contextualize_q_system_prompt = (

    """
      Given a chat history and the latest user question,
      which might reference context in the chat history ,
      formulate a standalone question which can be understood,
      without the chat history. Do not answer the question,
      just reformulate it if needed and otherwise return it as is.
    """

)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)


In [34]:
history_aware_retriever = create_history_aware_retriever(
    llm,
    retriever,
    contextualize_q_prompt,
)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7a5dd4a0e920>, search_kwargs={'k': 5}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMess

In [35]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

In [36]:
question_answer_chain = create_stuff_documents_chain(
    llm,
    prompt=qa_prompt,
)

In [38]:
rag_chain = create_retrieval_chain(
    history_aware_retriever,
    question_answer_chain,
)
from langchain_core.messages import AIMessage, HumanMessage
chat_history = []

question = "What is Self-Refection?"
response1 = rag_chain.invoke(
    {
        "input": question,
        "chat_history": chat_history
    }
)
chat_history.extend([HumanMessage(content=question), AIMessage(content=response1['answer'])])

question2 = "Tell me more about it"

response2 = rag_chain.invoke(
    {
        "input": question2,
        "chat_history": chat_history
    }
)
print(response2['answer'])

<think>
Alright, let me try to work through this. The user is asking about Self-Reflection, and I need to provide a detailed explanation based on the information given. 

First, I recall that Self-Reflection is a crucial aspect for autonomous agents. It allows them to improve over time by reviewing past actions and mistakes. This is especially important in real-world scenarios where trial and error are common. 

Looking at the ReAct framework, it combines reasoning and acting, enabling the model to interact with the environment through specific actions and generate natural language reasoning. The prompt template includes sections like Thought, Action, Observation, which are repeated multiple times. This structured approach helps the model systematically think through problems, take actions, and learn from the outcomes.

The user previously asked, "What is Self-Refection?" and I explained that it's a process for agents to learn from past experiences. Now, the user is asking, "Tell me mo

In [39]:
chat_history

[HumanMessage(content='What is Self-Refection?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="<think>\nOkay, so I need to figure out what Self-Reflection is based on the provided information. Let me go through the text step by step.\n\nFirst, I see that Self-Reflection is mentioned as a vital aspect for autonomous agents. It allows them to improve by refining past decisions and correcting mistakes. That makes sense because in real-world tasks, trial and error are common, so being able to reflect would help in learning from those errors.\n\nLooking further, there's a mention of ReAct (Yao et al. 2023) which integrates reasoning and acting within LLMs. The prompt template includes steps like Thought, Action, Observation, repeated multiple times. This structure probably helps the model think through each step, take actions, observe the results, and then reflect on what happened.\n\nThe text also talks about a Reflexion framework where self-reflection is created by show

In [42]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id : str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversation_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    history_messages_key="chat_history",
    input_message_key="input",
    output_message_key="answer",
)


conversation_rag_chain.invoke(
    {"input": "What is Self-Refection?",},
        config={
            "configurable": {
                "session_id": "session1"
            }
        }
)['answer']

Error in RootListenersTracer.on_chain_end callback: KeyError('output')


"<think>\nAlright, I need to figure out what Self-Reflection is based on the provided information. Let me read through the text again to understand the key points.\n\nFirst, the Reflexion framework is mentioned, which involves a heuristic function to stop inefficient or hallucinatory trajectories. It defines inefficient planning as taking too long without success and hallucination as repeated identical actions leading to the same observation. The self-reflection process uses two-shot examples, each being a pair of a failed trajectory and an ideal reflection. These reflections are stored in the agent's working memory, up to three, to guide future planning.\n\nThen, the text discusses Self-Reflection as a vital aspect for autonomous agents to improve by refining past decisions and correcting mistakes. It's important in real-world tasks where trial and error are common. The ReAct framework is mentioned, integrating reasoning and acting within LLMs by combining discrete actions and languag

In [43]:
conversation_rag_chain.invoke(
    {"input": "What are common ways of doing it ?",},
        config={
            "configurable": {
                "session_id": "session1"
            }
        }
)['answer']

Error in RootListenersTracer.on_chain_end callback: KeyError('output')


"<think>\nOkay, so I'm trying to figure out how to answer the question about common ways of task decomposition in AI systems. The user provided some resources and the question is about the methods used. \n\nFirst, I remember from the resources that there's something called Chain of Thought (CoT) by Wei et al. in 2022. CoT is a prompting technique where the model is instructed to think step by step. This helps break down complex tasks into simpler steps, making it easier to handle. It uses more computation at test time to do this decomposition.\n\nThen, there's the Tree of Thoughts (ToT) by Yao et al. in 2023. This extends CoT by not just stopping at a single chain but exploring multiple reasoning paths at each step. It creates a tree structure where each node can branch out into several possibilities. The system can use BFS or DFS to search through these possibilities and evaluate each state, maybe using a classifier or majority vote to decide the best path.\n\nI'm also thinking about 